# 🧼 Silver : `clean_sales_enriched`  
## Enrichissement des ventes avec dimensions, calculs de base, détection d'anomalies  
## → Table prête pour agrégation & diagnostic

In [ ]:
from pyspark.sql import SparkSession

# -----------------------------------------------------
# 1. Démarrage de la session Spark
# -----------------------------------------------------
spark = SparkSession.builder.appName("IcebergNotebook").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
print("Spark session initialisée avec Iceberg et MinIO.")

# 🗃️ Création du namespace silver
spark.sql('CREATE NAMESPACE IF NOT EXISTS retail.silver').show()
print(f"Namespace Iceberg créée : silver")

In [ ]:
spark.sql("""
CREATE OR REPLACE TABLE retail.silver.clean_sales_enriched
COMMENT 'Ventes nettoyées + enrichies — couche Silver'
AS
WITH base AS (
  SELECT
    s.sale_id,
    s.sale_date,
    s.store_id,
    st.name AS store_name,
    st.city,
    st.country,
    s.product_id,
    p.name AS product_name,
    p.category,
    p.brand,
    p.cost_price,
    p.list_price,
    s.quantity,
    s.unit_price,
    s.unit_price * s.quantity AS revenue,
    p.cost_price * s.quantity AS cost,
    (s.unit_price - p.cost_price) * s.quantity AS gross_profit,
    s.unit_price < p.list_price AS is_discounted,
    (p.list_price - s.unit_price) / p.list_price * 100 AS discount_applied_pct,
    e.employee_id,
    CONCAT(e.first_name, ' ', e.last_name) AS salesperson,
    e.job_title
  FROM retail.raw.sales s
  JOIN retail.raw.stores st ON s.store_id = st.store_id
  JOIN retail.raw.products p ON s.product_id = p.product_id
  JOIN retail.raw.employees e ON s.employee_id = e.employee_id
)
SELECT *,
  -- 📌 Indicateurs Silver (calculs stables, non agrégés)
  CASE 
    WHEN gross_profit < 0 THEN 'Perte'
    WHEN gross_profit = 0 THEN 'Seuil'
    ELSE 'Profit'
  END AS margin_status,
  
  -- 🚩 Détection anomalies (règles métier simples)
  CASE 
    WHEN quantity > 10 THEN 'Volume élevé'
    WHEN discount_applied_pct > 30 THEN 'Promo forte'
    WHEN unit_price < cost_price * 0.8 THEN 'Prix anormalement bas'
    ELSE 'Normal'
  END AS alert_flag

FROM base
""")

print("✅ Table Silver créée : retail.silver.clean_sales_enriched")
spark.sql("SELECT * FROM retail.silver.clean_sales_enriched LIMIT 3").show(truncate=False)